In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
import pandas as pd

data = pd.DataFrame(heart_disease.data.original) 
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


Primeiramente mostrando que o nosso problema de classificação se encaixa num problema multiclasse

In [2]:
data['num'].value_counts()

num
0    164
1     55
2     36
3     35
4     13
Name: count, dtype: int64

5 classes diferentes para cada instância, representando:<br>
    - 0 para nenhuma doença cardíaca<br>
    - 1 a 4 para diferentes doenças cardíacas

DADOS DESBALANCEADOS !!!

# Apenas tratando valores ausentes!!

In [3]:
from sklearn.impute import SimpleImputer
import numpy as np

dtypes = data.dtypes
columns = data.columns # Salvando as colunas da base de dados

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
data = imp.fit_transform(data)
data = pd.DataFrame(data, columns=columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    float64
 12  thal      303 non-null    float64
 13  num       303 non-null    float64
dtypes: float64(14)
memory usage: 33.3 KB


Apenas separando em x e y, também separando conjunto de treinamento e de teste

In [4]:
from sklearn.model_selection import train_test_split

# data (as pandas dataframes) 
y = data['num']
x = data.drop(columns=['num'])
print(x.head())

# x_train, x_test, y_train, y_test = train_test_split(
#     x, 
#     y, 
#     test_size=0.3, 
#     random_state=0, 
#     shuffle=True, 
#     stratify=y
# )

    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   

   slope   ca  thal  
0    3.0  0.0   6.0  
1    2.0  3.0   3.0  
2    2.0  2.0   7.0  
3    3.0  0.0   3.0  
4    1.0  0.0   3.0  


# Desempenho de algoritmos inerentemente multiclasse

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

metrics = []
models = {
    "Tree": DecisionTreeClassifier(random_state=0),
    "Bagging": BaggingClassifier(random_state=0),
    "AdaBoosting": AdaBoostClassifier(random_state=0),
    "RandomForest": RandomForestClassifier(random_state=0)
}


for model_name, model in models.items():

    # Estratificando conjunto de dados
    skf = StratifiedKFold(n_splits=5) # 5 pois o conjunto de dados é desbalanceado

    y_pred = cross_val_predict(model, x, y, cv=skf)

     # Calcular as métricas
    accuracy = accuracy_score(y, y_pred)
    f1_macro = f1_score(y, y_pred, average='macro')
    f1_weighted = f1_score(y, y_pred, average='weighted')
    
    # Armazenar os resultados em uma lista
    metrics.append({
        "Modelo": model_name,
        "Acurácia": accuracy,
        "F1 Macro": f1_macro,
        "F1 Weighted": f1_weighted
    })

df_metrics = pd.DataFrame(metrics)
print(df_metrics)

/home/dani/ciencias-da-computacao/8-semestre/mineracao-de-dados/atividade-1/venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dani/ciencias-da-computacao/8-semestre/mineracao-de-dados/atividade-1/venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dani/ciencias-da-computacao/8-semestre/mineracao-de-dados/atividade-1/venv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dani/ciencias-da-computacao/8-semestre/mineracao-de-

         Modelo  Acurácia  F1 Macro  F1 Weighted
0          Tree  0.495050  0.293748     0.496732
1       Bagging  0.528053  0.273287     0.504783
2   AdaBoosting  0.498350  0.271768     0.500225
3  RandomForest  0.580858  0.292685     0.531037


Melhor algoritmo para a nossa base de dados: RandomForest !!

# Tunando hiper-parâmetros de um RandomForest

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

rfc = RandomForestClassifier(random_state=0)

# Definindo os hiperparâmetros para o Grid Search
param_grid = {
    'n_estimators': [5, 10, 20, 50],  # Número de árvores
    'max_depth': [None, 10, 20, 30],  # Profundidade máxima
    'min_samples_split': np.arange(2, 11),  # Split mínimo
    'min_samples_leaf': np.arange(1, 5),  # Folhas mínimas
    'max_features': [None, 'sqrt', 'log2']  # Máximos recursos para split
}

# Usando múltiplas métricas
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_macro': make_scorer(f1_score, average='macro'),
    'f1_weighted': make_scorer(f1_score, average='weighted')
}


grid_search = GridSearchCV(
    rfc, 
    param_grid=param_grid,
    scoring=scoring,
    cv=skf,
    refit='f1_macro', # por se tratar de um dataset desbalanceado, não usamos média ponderada
    n_jobs=-1
)
# Ajustar o GridSearchCV
model_rf = grid_search.fit(x, y) 

print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=10, min_samples_leaf=np.int64(1),
                       min_samples_split=np.int64(6), n_estimators=10,
                       random_state=0)


In [8]:
# Extrair os resultados da grid_search
results = grid_search.cv_results_

# Criar o DataFrame com as colunas desejadas
df_results = pd.DataFrame({
    'Parâmetros': results['params'],
    'Acurácia (Média)': results['mean_test_accuracy'],
    'Acurácia (Desvio)': results['std_test_accuracy'],
    'F1 Macro (Média)': results['mean_test_f1_macro'],
    'F1 Macro (Desvio)': results['std_test_f1_macro'],
    'F1 Weighted (Média)': results['mean_test_f1_weighted'],
    'F1 Weighted (Desvio)': results['std_test_f1_weighted']
})

# Exibir o DataFrame para visualizar os resultados
print(df_results)

                                             Parâmetros  Acurácia (Média)  \
0     {'max_depth': None, 'max_features': None, 'min...          0.547760   
1     {'max_depth': None, 'max_features': None, 'min...          0.540984   
2     {'max_depth': None, 'max_features': None, 'min...          0.577268   
3     {'max_depth': None, 'max_features': None, 'min...          0.560710   
4     {'max_depth': None, 'max_features': None, 'min...          0.570710   
...                                                 ...               ...   
1723  {'max_depth': 30, 'max_features': 'log2', 'min...          0.600492   
1724  {'max_depth': 30, 'max_features': 'log2', 'min...          0.580874   
1725  {'max_depth': 30, 'max_features': 'log2', 'min...          0.564317   
1726  {'max_depth': 30, 'max_features': 'log2', 'min...          0.577486   
1727  {'max_depth': 30, 'max_features': 'log2', 'min...          0.593934   

      Acurácia (Desvio)  F1 Macro (Média)  F1 Macro (Desvio)  \
0          

In [9]:
print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

RandomForestClassifier(max_depth=10, min_samples_leaf=np.int64(1),
                       min_samples_split=np.int64(6), n_estimators=10,
                       random_state=0)
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(6), 'n_estimators': 10}
0.35711714007383877


In [11]:
best_params = grid_search.best_params_

rf_best = RandomForestClassifier(**best_params, random_state=0)

# Usar validação cruzada para obter previsões com cross_val_predict
skf = StratifiedKFold(n_splits=5)
y_pred = cross_val_predict(rf_best, x, y, cv=skf)

# Avaliar o modelo com as métricas desejadas
accuracy = accuracy_score(y, y_pred)
f1_macro = f1_score(y, y_pred, average='macro')
f1_weighted = f1_score(y, y_pred, average='weighted')

metrics.append({
        "Modelo": "Tuned RandomForest",
        "Acurácia": accuracy,
        "F1 Macro": f1_macro,
        "F1 Weighted": f1_weighted
})

df_metrics = pd.DataFrame(metrics)
print(df_metrics)

               Modelo  Acurácia  F1 Macro  F1 Weighted
0                Tree  0.495050  0.293748     0.496732
1             Bagging  0.528053  0.273287     0.504783
2         AdaBoosting  0.498350  0.271768     0.500225
3        RandomForest  0.580858  0.292685     0.531037
4  Tuned RandomForest  0.617162  0.375294     0.577266


A tunagem de hiper-parâmetros melhorou a nossa RandomForest em todas as métricas, entretanto, esperava mais, a busca de hiper-parâmetros demorou 3 minutos e mais de 1500 modelos foram testados

# OVO com Árvore de decisão

In [12]:
from sklearn.multiclass import OneVsOneClassifier

ovo_tree = OneVsOneClassifier(
    DecisionTreeClassifier(random_state=0)
)

# Usar validação cruzada para obter previsões com cross_val_predict
skf = StratifiedKFold(n_splits=5)
y_pred = cross_val_predict(ovo_tree, x, y, cv=skf)

# Avaliar o modelo com as métricas desejadas
accuracy = accuracy_score(y, y_pred)
f1_macro = f1_score(y, y_pred, average='macro')
f1_weighted = f1_score(y, y_pred, average='weighted')

metrics.append({
        "Modelo": "OneVsOne",
        "Acurácia": accuracy,
        "F1 Macro": f1_macro,
        "F1 Weighted": f1_weighted
})

df_metrics = pd.DataFrame(metrics)
print(df_metrics)

               Modelo  Acurácia  F1 Macro  F1 Weighted
0                Tree  0.495050  0.293748     0.496732
1             Bagging  0.528053  0.273287     0.504783
2         AdaBoosting  0.498350  0.271768     0.500225
3        RandomForest  0.580858  0.292685     0.531037
4  Tuned RandomForest  0.617162  0.375294     0.577266
5            OneVsOne  0.521452  0.339700     0.536474


In [13]:
from sklearn.multiclass import OneVsRestClassifier

ova_tree = OneVsRestClassifier(
    DecisionTreeClassifier(random_state=0)
)

# Usar validação cruzada para obter previsões com cross_val_predict
skf = StratifiedKFold(n_splits=5)
y_pred = cross_val_predict(ova_tree, x, y, cv=skf)

# Avaliar o modelo com as métricas desejadas
accuracy = accuracy_score(y, y_pred)
f1_macro = f1_score(y, y_pred, average='macro')
f1_weighted = f1_score(y, y_pred, average='weighted')

metrics.append({
        "Modelo": "OneVsAll",
        "Acurácia": accuracy,
        "F1 Macro": f1_macro,
        "F1 Weighted": f1_weighted
})

df_metrics = pd.DataFrame(metrics)
print(df_metrics)

               Modelo  Acurácia  F1 Macro  F1 Weighted
0                Tree  0.495050  0.293748     0.496732
1             Bagging  0.528053  0.273287     0.504783
2         AdaBoosting  0.498350  0.271768     0.500225
3        RandomForest  0.580858  0.292685     0.531037
4  Tuned RandomForest  0.617162  0.375294     0.577266
5            OneVsOne  0.521452  0.339700     0.536474
6            OneVsAll  0.369637  0.225109     0.412288


A RandomForest tunada ainda foi a melhor estratégia visando o nosso dataset
<br>
<br>
OVO se saiu considerávelmente melhor do que OVA em todas as medidas.